V1-V22 uses the lightgbm model, with the best version being V11.

From V23， try other models

reference: https://www.kaggle.com/jiaoyouzhang

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

OrdialEncoder: 문자(카테고리)데이터를 숫자로 ('S','M' -> 0,1)  
StandardScalar: 데이터 단위 통일 (평균 0, 분산 1)

torch: PyTorch  
torch.nn as nn: 신경망의 layer와 구조를 만듬  
optim: 최적화 도구 (Adam, SGD)  
TensorDataset: 입력 데이터와 정답을 하나로 묶어줌  
DataLoader: 묶인 데이터를 학습하기 좋게 batch  

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


train_file = "/kaggle/input/playground-series-s6e1/train.csv"
test_file = "/kaggle/input/playground-series-s6e1/test.csv"
original_file = "/kaggle/input/exam-score-prediction-dataset/Exam_Score_Prediction.csv"

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
original_df = pd.read_csv(original_file) 
submission_df = pd.read_csv("/kaggle/input/playground-series-s6e1/sample_submission.csv") 
TARGET = 'exam_score'

num_features = ['study_hours', 'class_attendance', 'sleep_hours']
base_features = [col for col in train_df.columns if col not in [TARGET, 'id']]
CATS = base_features
NUMS = num_features  # only these are truly numerical

현재 컴퓨터에 CUDA가 잇는지 확인하고 있으면 CUDA, 없으면 CPU 선택  

trian, test, original file을 가져옴

TARGET변수 목표를 넣고,  
num_features에 숫자 데이터  
base_features에 모든 컬럼 (target, id 제외)  

CATS: 범주형  
NUMS: 숫자형  

In [ ]:
def add_engineered_features(df):
    df_temp = df.copy()
    # Sine features
    df_temp['_study_hours_sin'] = np.sin(2 * np.pi * df_temp['study_hours'] / 12).astype('float32')
    df_temp['_class_attendance_sin'] = np.sin(2 * np.pi * df_temp['class_attendance'] / 12).astype('float32')

    for col in num_features:
        if col in df_temp.columns:
            df_temp[f'log_{col}'] = np.log1p(df_temp[col])
            df_temp[f'{col}_sq'] = df_temp[col] ** 2

    for col in train_df.select_dtypes(include=['object','category']).columns.tolist():
        cat_series = df_temp[col].astype(str)
        freq_map = cat_series.value_counts().to_dict()
        df_temp[f"{col}_freq"] = cat_series.map(freq_map).fillna(0).astype(int)
        
    # Linear combo feature
    df_temp['feature_formula'] = (
            5.9051154511950499 * df_temp['study_hours'] +
            0.34540967058057986 * df_temp['class_attendance'] +
            1.423461171860262 * df_temp['sleep_hours'] + 4.7819
    )

    # Keep categorical as string for encoding
    for col in CATS:
        df_temp[col] = df_temp[col].astype(str)

    return df_temp

새로운 데이터 만드는 Feature Engineering  

_study_hour_sin이라는 새로운 컬럼을 만들음 
공부를 하는 시간에 다른 패턴이 있나? (0-12시간을 한 주기로 보고)  
실험적인 의도라고 생각함  

for col in num_features: 수치형 데이터 변환(로그 & 제곱)  
수치형 데이터는 각각 log와 제곱 컬럼을 추가  

for col in train_df.select_dtypes  
원본 학습 데이터(trian_df)에서 글자(object)나 범주(cat)인 컬럼  
cat_series => 해당 열을 확실하게 문자열로  
freq_map => 해당 열에서 각 항목이 몇번 나왔는지 dict로 변환  
{col}_freq 컬럼에 각 컬럼의 빈도수를 숫자로 바꿔서 저장, 빈칸은 0으로 채우고 null로 변환  
ex) A학교 -> 50, B학교 -> 10, A학교 -> 50  

feature_formula: 작성자가 이 모델을 만들기 전에 linear regression같은  
단순한 모델을 먼저 돌려엇 얻은 최적의 계수  

for col in CATS:  
범주형 데이터를 문자열로 고정하고 return  

In [ ]:
train_eng = add_engineered_features(train_df)

all_num_cols = [col for col in train_eng.columns if col not in CATS + [TARGET, 'id']]
all_cat_cols = CATS

scaler = StandardScaler()
scaler.fit(train_eng[all_num_cols])

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(train_eng[all_cat_cols])

train_eng에 train_df를 이용해 위의 함수를 이용해 가공한 데이터 넣음  
all_num_cols, all_cat_cols로 더 만들어진 컬럼을 다시 분류  

StandardScaler: 각 컬럼에서 평균과 표준편차를 학습  
fit으로 계산해서 기억해 놓음  

OrdinalEncoder: CAT를 숫자로 바꿔줌  
handle_unknown, unknown_values=-1 학습데이터에 없는 데이터가 test 데이터에서 갑자기 나오면 -1로 표시하고 넘어감  
fit으로 각각 CAT에 맞는 번호를 만들어 놓음 (A학교는 0, B학교는 1)

In [ ]:
def preprocess_pipeline_separate(df):
    df_eng = add_engineered_features(df)
    # Numerical: scale
    nums_scaled = scaler.transform(df_eng[all_num_cols])
    # Categorical: encode to integers
    cats_encoded = encoder.transform(df_eng[all_cat_cols]).astype(np.int64)
    return nums_scaled, cats_encoded

앞서 만들어둔 Feature_Engineering, Scaler, Encoder을 사용해  
실제 데이터를 가공 // nums 와 cats 컬럼을 따로 분류  

나중에 test data에서 사용가능, fit은 train에서 한걸로 (data leakage)  

In [ ]:
X_num, X_cat = preprocess_pipeline_separate(train_df)
y = train_df[TARGET].values
X_test_num, X_test_cat = preprocess_pipeline_separate(test_df)
X_orig_num, X_orig_cat = preprocess_pipeline_separate(original_df)
y_original = original_df[TARGET].values

cat_unique_counts = []
for i, col in enumerate(all_cat_cols):
    n_unique = int(encoder.categories_[i].size)
    cat_unique_counts.append(n_unique)

print("Categorical feature cardinalities:", cat_unique_counts)

Scikit-Learn 도구들은 기본적으로 입력이 DataFrame이여도  
출력은 Numpy array  
위 데이터들은 numpy array들임  

train_df: AI가 생성한 가짜 데이터
original_df: 그 가짜를 만들 때 참고했던 진짜 원본 데이터

cat_unique_counts 임베딩 크기 계산  
각 범주형 변수마다 몇 종류의 데이터가 있는지 저장 (남여 2개 인지,  
a,b,c 3개인지 등등  

In [ ]:
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=4):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (batch, channels)
        se = x.mean(dim=0, keepdim=True)  # global avg pool -> (1, channels)
        se = self.fc1(se)
        se = self.relu(se)
        se = self.fc2(se)
        se = self.sigmoid(se)
        return x * se  # broadcast

SEBlock: 어떤 feature가 중요한지 스스로 공부해서, 중요한 건 키우고  
중요하지 않는건 끄는 역할  

nn.Module: PyTorch 모델이 갖춰야할 모든 기본 소양(파라미터 관리,  
GPU이동, 저장/로드 등)을 미리 만들어 둔 기본 틀  
이 틀을 가져다가 그 안에 원하는 함수 선언  

channels: 입력 데이터 갯수, reduction: 압축, 기본값 4  
fc1 => 입력 channels/reduction 만큼 줄임 (알아서 학습해서 줄임)  
relu => 음수는 0  
fc2 => 줄어든걸 다시 원상 복귀  
sigmoid => 숫자를 0~1사이로 바꿈 (각 특징의 중요도를 점수로)  

se = x.mean => 들어온 데이터를 세로 방향(dim=0)으로 평균 냄  
ex) (10,20) -> (1,20)  

se값이 높으면 정보가 통과하고, 낮으면 차단됨  중요도가 높은 것만 남음  

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.1, reduction=4):
        super().__init__()
        self.linear1 = nn.Linear(dim, dim)
        self.linear2 = nn.Linear(dim, dim)
        self.dropout = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.se = SEBlock(dim, reduction=reduction)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        # First sub-block
        out = self.norm1(x)
        out = self.linear1(out)
        out = self.relu(out)
        out = self.dropout(out)
        # Second sub-block
        out = self.norm2(out)
        out = self.linear2(out)
        out = self.dropout(out)
        # SE
        out = self.se(out)
        # Residual connection
        out = out + residual
        return out

residual block: 원본 + residual  

Linear(dim, dim) 차원을 줄이지 않고 정보를 조합하여 더 좋은 정보를 만듬

Dropout -> 학습할때 뇌세포의 10%(0.1)을 랜덤으로 꺼버림  
특정 뉴런 하나에 의존하지 말고 다 같이 이용하라는 뜻 (과적합 방지) 
ex) ReLU에서 나온 데이터를 10%는 꺼지게 함  

LayerNorm -> 데이터가 레이어를 지날 때마다 값이 너무 커지거나 작아지지  
않게, 평균 0, 분산 1로 계속 맞춰줌, 학습의 안정성  

In [ ]:
class TabularResNetWithEmbedding(nn.Module):
    def __init__(
            self,
            num_numerical,
            cat_unique_counts,
            embedding_dim=8,
            hidden_dim=256,
            n_blocks=4,
            dropout=0.1,
            head_dims=[64, 16]
    ):
        super().__init__()
        self.num_numerical = num_numerical
        self.embedding_dim = embedding_dim

        # Embedding layers for each categorical feature
        # ModuleList: 파이토치 전용 리스트
        # nn.Embedding: 단어장, 
        # cat에서 몇개 종류가 있는지 계산했던 걸 바탕으로 embedding_dim에 만들어줌
        # ex) A학교가 1이였는데, [-2.2, 0.1 ... ]등 8개 숫자를 갖는 차원으로 바꿔줌
        # 처음에는 랜덤이지만 AI가 학습
        self.embeddings = nn.ModuleList([
            nn.Embedding(n_cat + 1, embedding_dim, padding_idx=-1)  # -1 mapped to last index
            for n_cat in cat_unique_counts
        ])

        total_cat_dim = len(cat_unique_counts) * embedding_dim
        # input_dim은 원래 있던 숫자 데이터 개수 + embedding을 거쳐 숫자로 변신한
        # 범주형 데이터들의 총 길이
        input_dim = num_numerical + total_cat_dim
        

        # Projection to hidden_dim
        # 입력 데이터가 몇개든 상관없이 모델 내부에서 사용할 크기로 통일 (256)
        self.proj = nn.Linear(input_dim, hidden_dim)
        # 데이터를 모델에 넣기 전에 일부러 훼손, 특정 변수 의존도 낮추기, 실전 데이터 노이즈 방지
        self.dropout_in = nn.Dropout(dropout)

        # Residual blocks
        # ResidualBlock을 n_blocks개(4개) 만큼 쌓아 올림
        self.blocks = nn.Sequential(
            *[ResidualBlock(hidden_dim, dropout=dropout) for _ in range(n_blocks)]
        )

        # Prediction head
        # 최종 256가지를 정리해서 점수를 내뱉는 (1개 output)과정
        layers = []
        prev = hidden_dim
        for h in head_dims:
            layers.extend([
                nn.Linear(prev, h),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev = h
        layers.append(nn.Linear(prev, 1))
        self.head = nn.Sequential(*layers)

    def forward(self, x_num, x_cat):
        # 입력을 2개로 받아 따로 처리
        # x_num: (B, num_numerical)
        # x_cat: (B, n_cats)
        batch_size = x_num.size(0)

        # Embed categorical features
        x_embeds = []
        # 범주형 데이터 변환
        for i, emb in enumerate(self.embeddings):
            # x_cat[:, i] shape: (B,)
            # i번째 범주형 변수만 꺼냄
            xi = x_cat[:, i]
            # Handle -1 (unknown): map to last embedding index
            # 모르는 값 -1 처리
            xi = torch.where(xi == -1, torch.tensor(emb.num_embeddings - 1, device=xi.device), xi)
            embed_i = emb(xi)  # (B, embedding_dim)
            # 단어장을 찾아서 숫자로 변환
            x_embeds.append(embed_i)

        # 변환된 범주형 데이터를 옆으로 이어 붙이고, 수치형 데이터도 붙임
        x_cat_emb = torch.cat(x_embeds, dim=1)  # (B, total_cat_dim)
        # Concat numerical and embedded categorical
        x = torch.cat([x_num, x_cat_emb], dim=1)  # (B, input_dim)

        # Project to hidden space
        # 규격 맞추고 dropout 10%
        x = self.proj(x)
        x = self.dropout_in(x)

        # Residual blocks
        # residual block 4번 통과
        x = self.blocks(x)

        # Prediction head
        # (n,1) 차원 n으로 펴서 return
        out = self.head(x).squeeze(1)
        return out

In [ ]:


def train_model(model, train_loader, val_loader, epochs=200, lr=1e-3, weight_decay=1e-5, patience=20, factor=0.5,
                min_lr=1e-6):
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=factor, patience=patience // 2, min_lr=min_lr
    )
    criterion = nn.MSELoss()

    best_val_loss = float('inf')
    patience_counter = 0
    best_weights = None

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for xb_num, xb_cat, yb in train_loader:
            xb_num, xb_cat, yb = xb_num.to(device), xb_cat.to(device), yb.to(device)
            optimizer.zero_grad()
            pred = model(xb_num, xb_cat)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb_num, xb_cat, yb in val_loader:
                xb_num, xb_cat, yb = xb_num.to(device), xb_cat.to(device), yb.to(device)
                pred = model(xb_num, xb_cat)
                loss = criterion(pred, yb)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        val_rmse = val_loss ** 0.5
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_rmse = val_rmse  # ← 记录 best RMSE
            patience_counter = 0
            best_weights = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

        # 👇 每 10 个 epoch 打印一次验证 RMSE
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch + 1}/{epochs} | Val RMSE: {val_rmse:.5f}")

    if best_weights is not None:
        model.load_state_dict(best_weights)
    return model, best_val_rmse


# ==============================
# K 折训练
# ==============================
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

test_predictions = []
oof_predictions = np.zeros(len(y))

print(f"Starting {n_splits}-fold CV with advanced TabularResNet...")

for fold, (train_idx, val_idx) in enumerate(kf.split(X_num, y)):
    print(f"\n--- Fold {fold + 1}/{n_splits} ---")

    # Split
    X_num_train, X_cat_train = X_num[train_idx], X_cat[train_idx]
    y_train = y[train_idx]
    X_num_val, X_cat_val = X_num[val_idx], X_cat[val_idx]
    y_val = y[val_idx]

    # Augment with original data
    X_num_combined = np.vstack([X_num_train, X_orig_num])
    X_cat_combined = np.vstack([X_cat_train, X_orig_cat])
    y_combined = np.concatenate([y_train, y_original])

    # Tensors
    X_num_train_t = torch.tensor(X_num_combined, dtype=torch.float32)
    X_cat_train_t = torch.tensor(X_cat_combined, dtype=torch.int64)
    y_train_t = torch.tensor(y_combined, dtype=torch.float32)

    X_num_val_t = torch.tensor(X_num_val, dtype=torch.float32)
    X_cat_val_t = torch.tensor(X_cat_val, dtype=torch.int64)
    y_val_t = torch.tensor(y_val, dtype=torch.float32)

    X_test_num_t = torch.tensor(X_test_num, dtype=torch.float32)
    X_test_cat_t = torch.tensor(X_test_cat, dtype=torch.int64)

    # Datasets & Loaders
    train_ds = TensorDataset(X_num_train_t, X_cat_train_t, y_train_t)
    val_ds = TensorDataset(X_num_val_t, X_cat_val_t, y_val_t)
    train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=1024, shuffle=False)

    # Model
    model = TabularResNetWithEmbedding(
        num_numerical=X_num.shape[1],
        cat_unique_counts=cat_unique_counts,
        embedding_dim=8,
        hidden_dim=256,
        n_blocks=3,
        dropout=0.11,
        head_dims=[64, 16]
    ).to(device)

    # Train
    model, best_rmse = train_model(
        model,
        train_loader,
        val_loader,
        epochs=300,
        lr=1e-3,
        weight_decay=1e-4,
        patience=20,
        factor=0.5,
        min_lr=1e-6
    )

    # Predict
    model.eval()
    with torch.no_grad():
        val_pred = model(X_num_val_t.to(device), X_cat_val_t.to(device)).cpu().numpy()
        test_pred = model(X_test_num_t.to(device), X_test_cat_t.to(device)).cpu().numpy()

    oof_predictions[val_idx] = val_pred
    test_predictions.append(test_pred)

    print(f"Fold {fold + 1} RMSE: {best_rmse:.5f}")

# ==============================
# Final Evaluation & Submission
# ==============================
oof_rmse = np.sqrt(mean_squared_error(y, oof_predictions))
print("\n" + "=" * 50)
print(f"Final OOF RMSE: {oof_rmse:.6f}")
print("=" * 50)

oof_df = pd.DataFrame({'id': train_df['id'], TARGET: oof_predictions})
oof_df.to_csv('nn_oof.csv', index=False)

submission_df[TARGET] = np.mean(test_predictions, axis=0)
submission_df.to_csv('submission.csv', index=False)
print("\nSubmission saved!")